In [ ]:
#  Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
"""Convolutional Neural Network Estimator for MNIST, built with tf.layers."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)


def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  # MNIST images are 28x28 pixels, and have one color channel
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  # Computes 32 features using a 5x5 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 28, 28, 1]
  # Output Tensor Shape: [batch_size, 28, 28, 32]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 28, 28, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 32]
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2
  # Computes 64 features using a 5x5 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 14, 14, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 64]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #2
  # Second max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 14, 14, 64]
  # Output Tensor Shape: [batch_size, 7, 7, 64]
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 7, 7, 64]
  # Output Tensor Shape: [batch_size, 7 * 7 * 64]
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

  # Dense Layer
  # Densely connected layer with 1024 neurons
  # Input Tensor Shape: [batch_size, 7 * 7 * 64]
  # Output Tensor Shape: [batch_size, 1024]
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

  # Add dropout operation; 0.6 probability that element will be kept
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 1024]
  # Output Tensor Shape: [batch_size, 10]
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


def main(unused_argv):
  # Load training and eval data
  mnist = tf.contrib.learn.datasets.load_dataset("mnist")
  train_data = mnist.train.images  # Returns np.array
  train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
  eval_data = mnist.test.images  # Returns np.array
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

  # Create the Estimator
  mnist_classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

  # Set up logging for predictions
  # Log the values in the "Softmax" tensor with label "probabilities"
  tensors_to_log = {"probabilities": "softmax_tensor"}
  logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)

  # Train the model
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=100,
      num_epochs=None,
      shuffle=True)
  mnist_classifier.train(
      input_fn=train_input_fn,
      steps=20000,
      hooks=[logging_hook])

  # Evaluate the model and print results
  eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": eval_data},
      y=eval_labels,
      num_epochs=1,
      shuffle=False)
  eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
  print(eval_results)

#  Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
"""Convolutional Neural Network Estimator for MNIST, built with tf.layers."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)


def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  # MNIST images are 28x28 pixels, and have one color channel
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  # Computes 32 features using a 5x5 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 28, 28, 1]
  # Output Tensor Shape: [batch_size, 28, 28, 32]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 28, 28, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 32]
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2
  # Computes 64 features using a 5x5 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 14, 14, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 64]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #2
  # Second max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 14, 14, 64]
  # Output Tensor Shape: [batch_size, 7, 7, 64]
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 7, 7, 64]
  # Output Tensor Shape: [batch_size, 7 * 7 * 64]
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

  # Dense Layer
  # Densely connected layer with 1024 neurons
  # Input Tensor Shape: [batch_size, 7 * 7 * 64]
  # Output Tensor Shape: [batch_size, 1024]
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

  # Add dropout operation; 0.6 probability that element will be kept
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 1024]
  # Output Tensor Shape: [batch_size, 10]
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


def main(unused_argv):
  # Load training and eval data
  mnist = tf.contrib.learn.datasets.load_dataset("mnist")
  train_data = mnist.train.images  # Returns np.array
  train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
  eval_data = mnist.test.images  # Returns np.array
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

  # Create the Estimator
  mnist_classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

  # Set up logging for predictions
  # Log the values in the "Softmax" tensor with label "probabilities"
  tensors_to_log = {"probabilities": "softmax_tensor"}
  logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)

  # Train the model
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=100,
      num_epochs=None,
      shuffle=True)
  mnist_classifier.train(
      input_fn=train_input_fn,
      steps=20000,
      hooks=[logging_hook])

  # Evaluate the model and print results
  eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": eval_data},
      y=eval_labels,
      num_epochs=1,
      shuffle=False)
  eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
  print(eval_results)


if __name__ == "__main__":
  tf.app.run()

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c26f014e0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorfl

INFO:tensorflow:loss = 2.3170187, step = 2
INFO:tensorflow:probabilities = [[0.10870275 0.09499353 0.11240425 0.09670727 0.09239185 0.10617566
  0.10747076 0.08938447 0.09482719 0.09694219]
 [0.09469759 0.1073093  0.1007062  0.10449903 0.10522822 0.10337617
  0.10026094 0.10791025 0.08697245 0.08903988]
 [0.09464122 0.10841856 0.1092185  0.08835022 0.10177258 0.10977627
  0.1002716  0.10099321 0.09567644 0.09088144]
 [0.10355074 0.11602549 0.10640494 0.08455808 0.09810195 0.10690811
  0.10389793 0.0798461  0.08478866 0.11591798]
 [0.10090047 0.10727924 0.1086717  0.09648246 0.09627861 0.10476805
  0.10675652 0.09330632 0.08486161 0.10069495]
 [0.10051887 0.10011361 0.10694154 0.08316493 0.09226263 0.11137009
  0.11051057 0.08582465 0.09132746 0.11796573]
 [0.10011013 0.09667183 0.1067429  0.08337748 0.11479307 0.10817729
  0.1012611  0.09449339 0.09213602 0.10223678]
 [0.10266269 0.09550209 0.11149979 0.08982458 0.10443158 0.08846757
  0.09494271 0.0953175  0.10065681 0.11669461]
 [0.1

INFO:tensorflow:global_step/sec: 5.18821
INFO:tensorflow:probabilities = [[0.08820572 0.10121291 0.09002241 0.08373263 0.11014093 0.10395177
  0.09452853 0.09611777 0.10181688 0.13027053]
 [0.10724764 0.09925859 0.09697665 0.08207989 0.09754392 0.11230899
  0.1150432  0.08719637 0.09041473 0.11192998]
 [0.10235281 0.10502983 0.10464701 0.10066921 0.11097579 0.10128396
  0.0984847  0.09209173 0.09197739 0.09248749]
 [0.10348441 0.09608238 0.10549642 0.09172622 0.10739699 0.10022676
  0.09665526 0.09844418 0.09730113 0.10318624]
 [0.1009978  0.10758223 0.10919066 0.09085414 0.10464837 0.0980058
  0.10194298 0.09389789 0.0901304  0.10274971]
 [0.09518763 0.109621   0.10017417 0.09843154 0.10437141 0.09979274
  0.10658564 0.09130472 0.09516487 0.09936634]
 [0.10736335 0.10353084 0.1085497  0.08847429 0.1010021  0.10965544
  0.08990613 0.08928958 0.08957986 0.11264873]
 [0.09592479 0.09573651 0.10128141 0.09747149 0.09610596 0.102607
  0.10796152 0.09180613 0.0998149  0.11129029]
 [0.099972

INFO:tensorflow:loss = 2.292907, step = 102 (19.275 sec)
INFO:tensorflow:probabilities = [[0.1189417  0.09498937 0.10477271 0.09984096 0.09774111 0.10585343
  0.09711425 0.0919461  0.08816664 0.1006338 ]
 [0.10648637 0.10769994 0.1049326  0.09000989 0.10728658 0.09057181
  0.10094369 0.09489289 0.09927064 0.09790561]
 [0.10844229 0.10282867 0.10794856 0.09203047 0.0869232  0.10834315
  0.10989812 0.086328   0.09468897 0.10256848]
 [0.10156155 0.10568219 0.10312746 0.0875389  0.10425836 0.11104213
  0.10003463 0.09330588 0.09723682 0.09621217]
 [0.11094631 0.09428459 0.10425048 0.09512209 0.10136916 0.10154279
  0.09561939 0.08245122 0.1030247  0.11138932]
 [0.09710867 0.1020484  0.1062765  0.09433372 0.0983945  0.10314354
  0.10388398 0.09638137 0.0924105  0.10601885]
 [0.09599859 0.10233219 0.11062387 0.09202908 0.09913891 0.10801732
  0.10016162 0.0965609  0.09943376 0.09570369]
 [0.09863244 0.10314786 0.10271498 0.08884332 0.10073928 0.10063586
  0.10338993 0.10136136 0.09485546 0.1

INFO:tensorflow:global_step/sec: 5.60687
INFO:tensorflow:probabilities = [[0.09311188 0.08777979 0.1071751  0.10445632 0.10904826 0.10625623
  0.09102977 0.09299424 0.09544212 0.11270639]
 [0.10779049 0.09037479 0.11451693 0.08316541 0.10486293 0.11261944
  0.10608115 0.08436462 0.09516917 0.10105508]
 [0.08597667 0.10586052 0.11200235 0.09154642 0.09567028 0.10122991
  0.10742821 0.08619434 0.10723126 0.10685997]
 [0.10393369 0.10200864 0.09936128 0.10618556 0.1043449  0.10428821
  0.10698967 0.08393708 0.08805651 0.10089447]
 [0.10240884 0.11052612 0.10960356 0.09208504 0.10064969 0.09678528
  0.10722142 0.08079987 0.09540513 0.10451502]
 [0.1131753  0.091898   0.10693513 0.10442793 0.0977241  0.09104507
  0.09549306 0.10384504 0.09892146 0.09653493]
 [0.11011285 0.09789661 0.12051934 0.09916076 0.09603868 0.1026532
  0.09362968 0.09236231 0.08682424 0.10080228]
 [0.10846142 0.09378307 0.10324146 0.09203749 0.10344112 0.0980773
  0.10353723 0.09497079 0.10423243 0.09821761]
 [0.09922

...

INFO:tensorflow:loss = 2.1956062, step = 602 (18.693 sec)
INFO:tensorflow:probabilities = [[0.1009294  0.08246341 0.13711798 0.11393514 0.09414324 0.0832223
  0.10179792 0.08584251 0.09994776 0.10060026]
 [0.11008971 0.08596243 0.10053504 0.10351112 0.08908308 0.11493085
  0.11836358 0.08134142 0.09925334 0.09692939]
 [0.20072043 0.06848757 0.12450833 0.08351732 0.07559201 0.08529059
  0.10713388 0.07957669 0.10000435 0.07516883]
 [0.10119023 0.1012299  0.10424201 0.09266225 0.11120543 0.09908592
  0.08893381 0.09862021 0.10509871 0.09773161]
 [0.10691999 0.06573254 0.12140235 0.11697411 0.09257317 0.09251668
  0.09084844 0.10489453 0.0937035  0.11443464]
 [0.09222442 0.08895923 0.11732827 0.11885995 0.09223863 0.09466036
  0.0992229  0.1142489  0.09332806 0.0889293 ]
 [0.14461115 0.07259004 0.11169451 0.09939266 0.06472671 0.09602587
  0.1167414  0.08780269 0.11984881 0.08656619]
 [0.09106531 0.10185442 0.08965927 0.09853421 0.09381384 0.10327037
  0.11529973 0.09604803 0.11328939 0.0

INFO:tensorflow:global_step/sec: 5.13841
INFO:tensorflow:probabilities = [[0.09479345 0.09886594 0.11123478 0.09748544 0.09092426 0.10162952
  0.10421392 0.09858033 0.11146291 0.09080944]
 [0.09660106 0.09548554 0.10562641 0.10491398 0.10569447 0.10232845
  0.11321042 0.08750816 0.09033881 0.09829265]
 [0.09344666 0.08665478 0.10106055 0.10655569 0.09993871 0.0892209
  0.0967389  0.12003098 0.11027216 0.09608065]
 [0.10052802 0.06592318 0.10158487 0.09825146 0.1010332  0.11644221
  0.08973339 0.13143879 0.09475956 0.10030534]
 [0.10314643 0.09620908 0.10308064 0.09100497 0.09871332 0.09232357
  0.08892752 0.11410476 0.09778479 0.11470497]
 [0.09411688 0.11099342 0.10570182 0.09841058 0.10207591 0.09594641
  0.10048683 0.08929709 0.10590598 0.09706505]
 [0.13366225 0.07952981 0.09946885 0.10487951 0.0930831  0.10600463
  0.0972503  0.07749266 0.098675   0.10995396]
 [0.10750801 0.08421928 0.09475607 0.09430112 0.08978613 0.09792689
  0.1184576  0.09681159 0.1097189  0.10651448]
 [0.1031

INFO:tensorflow:loss = 2.1708663, step = 702 (19.461 sec)
INFO:tensorflow:probabilities = [[0.09454493 0.07498127 0.1392542  0.0919432  0.10562792 0.09259812
  0.08872018 0.12102363 0.08467885 0.1066276 ]
 [0.10994782 0.07588451 0.0997403  0.11246316 0.09271289 0.0803961
  0.09992014 0.11687891 0.08878646 0.12326962]
 [0.13501336 0.06873089 0.14230004 0.11558718 0.07851727 0.08805224
  0.10022096 0.07273842 0.12832697 0.07051266]
 [0.11313378 0.07164754 0.10463256 0.08906643 0.10845918 0.10470428
  0.12003388 0.07960378 0.10518733 0.10353123]
 [0.11502956 0.08681181 0.10201669 0.10859959 0.10959891 0.10193505
  0.10288991 0.0841267  0.09678809 0.09220371]
 [0.11168592 0.07682132 0.10901531 0.09260461 0.11743542 0.09310646
  0.09133026 0.08557182 0.10103333 0.12139552]
 [0.10147713 0.07889624 0.12386694 0.13516398 0.08558127 0.08908875
  0.09889797 0.0847104  0.10303602 0.09928132]
 [0.09169699 0.08444139 0.13462469 0.10568453 0.08324504 0.09886838
  0.12074635 0.06940165 0.11534254 0.0

INFO:tensorflow:global_step/sec: 4.67339
INFO:tensorflow:probabilities = [[0.16283938 0.05911865 0.12665458 0.12840235 0.05952807 0.08071238
  0.10884154 0.08828749 0.09027087 0.09534461]
 [0.10537279 0.06947763 0.13896018 0.11011939 0.0865173  0.08259068
  0.103127   0.07068896 0.13550633 0.09763972]
 [0.09763748 0.11866681 0.10323341 0.09326077 0.08888257 0.09663651
  0.10236412 0.0961481  0.09320392 0.10996639]
 [0.10149514 0.08482236 0.10950504 0.09688517 0.09887125 0.098497
  0.11412814 0.08811296 0.11108556 0.09659731]
 [0.09230388 0.11909644 0.09988552 0.09244671 0.10155604 0.09694605
  0.09705627 0.0968238  0.09801617 0.10586912]
 [0.09796676 0.09012185 0.11201129 0.09614777 0.08329854 0.08870841
  0.15697147 0.08230507 0.08951849 0.10295036]
 [0.09250481 0.11717413 0.10921101 0.10845955 0.08534677 0.09555802
  0.09688575 0.09109509 0.1084868  0.0952782 ]
 [0.10841891 0.06894084 0.11101442 0.10004766 0.11224345 0.09082938
  0.09178231 0.11109421 0.09206061 0.11356826]
 [0.09742

INFO:tensorflow:loss = 2.1151757, step = 802 (21.403 sec)
INFO:tensorflow:probabilities = [[0.08799819 0.10536065 0.10055714 0.10585932 0.09640668 0.09508587
  0.12026127 0.09173098 0.10764273 0.0890971 ]
 [0.15680018 0.07661617 0.11662881 0.09261308 0.08898421 0.09558012
  0.11563098 0.10248189 0.0799557  0.07470889]
 [0.10217213 0.11525598 0.11089576 0.09925291 0.08874981 0.09765583
  0.10834034 0.07879386 0.10413419 0.09474915]
 [0.09314153 0.08639976 0.09325743 0.14490606 0.07163041 0.10688314
  0.09246824 0.12018846 0.09587526 0.09524972]
 [0.11931682 0.08541558 0.10225828 0.1046561  0.08486374 0.07817695
  0.1137091  0.08367183 0.13686797 0.0910637 ]
 [0.09181275 0.07129904 0.13369599 0.08838725 0.09033888 0.11241966
  0.11550845 0.07578012 0.10352358 0.11723434]
 [0.09752636 0.08980387 0.09136793 0.10329517 0.10822907 0.08774821
  0.10377713 0.11112903 0.08585959 0.12126368]
 [0.11016341 0.09027719 0.135583   0.10410457 0.07453839 0.08639361
  0.13468058 0.06451487 0.1252291  0.